# Soccer Analysis using PySpark
___

### Configurações de Ambiente
___

#### Install Dependencies

In [0]:
!pip install pandas pyspark --quiet

#### Imports e Configurações

In [0]:
import requests
import pandas as pd
from io import StringIO
from pyspark.sql import SparkSession

#### Setup

In [0]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [0]:
def get_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return StringIO(response.text)
    else:
        raise Exception(f"Falha ao baixar o arquivo: {response.status_code}")

In [0]:
spark = (
    SparkSession.builder 
    .appName("Soccer Analysis") 
    .getOrCreate()
)

csv_content = get_data("https://raw.githubusercontent.com/fabiogjardim/bases/refs/heads/master/results.csv")
csv = pd.read_csv(csv_content)

df = spark.createDataFrame(csv)
df.show()

In [0]:
df.printSchema()

### Perguntas
___

#### 1. Quantos registros existem na base?

In [0]:
total_registros = df.count()
print(f"{color.BOLD}{color.GREEN}RESPOSTA{color.END}\nA base de dados possui {total_registros} registros.")

#### 2. Quantas equipes únicas mandantes existem na base?

In [0]:
equipes_mandantes_unicas = df.select("home_teamName").distinct().count()
print(f"{color.BOLD}{color.GREEN}RESPOSTA{color.END}\nA base de dados possui {equipes_mandantes_unicas} registros únicos de equipes mandantes.")

#### 3. Quantas vezes as equipes mandantes saíram vitoriosas?

In [0]:
vitoria_mandante = df.filter(df["home_scoreHome"] > df["away_scoreAway"]).count()
print(f"{color.BOLD}{color.GREEN}RESPOSTA{color.END}\nA base de dados possui {vitoria_mandante} registros onde o time mandante venceu.")

#### 4. Quantas vezes as equipes visitantes saíram vitoriosas?

In [0]:
vitoria_visitante = df.filter(df["away_scoreAway"] > df["home_scoreHome"]).count()
print(f"{color.BOLD}{color.GREEN}RESPOSTA{color.END}\nA base de dados possui {vitoria_visitante} registros onde o time visitante venceu.")

#### 5. Quantas partidas resultaram em empate?

In [0]:
df = df.withColumn("is_draw", df["home_scoreHome"] == df["away_scoreAway"])
empates = df.filter(df["is_draw"]).count()
print(f"{color.BOLD}{color.GREEN}RESPOSTA{color.END}\nA base de dados possui {empates} registros onde houve empate.")

#### 6. Quantas partidas foram realizadas em cada país?

In [0]:
print(f"{color.BOLD}{color.GREEN}RESPOSTA{color.END}")
df.groupBy("countryCountry").count().orderBy("count", ascending=False).show()

#### 7. Qual país teve mais partidas?

In [0]:
most_matches_country = df.groupBy("countryCountry").count().orderBy("count", ascending=False).first()
print(f"{color.BOLD}{color.GREEN}RESPOSTA{color.END}\nO país com mais partidas é {most_matches_country['countryCountry']} com {most_matches_country['count']} partidas.")

#### 8. Qual a partida com maior número de gols?

In [0]:
df = df.withColumn("total_goals", df["home_scoreHome"] + df["away_scoreAway"])
maior_quantidade_gols = df.orderBy(df["total_goals"].desc()).first()

print(f"{color.BOLD}{color.GREEN}RESPOSTA{color.END}")
df.filter(df["total_goals"] == maior_quantidade_gols["total_goals"]).show()

#### 9. Qual a maior goleada?

In [0]:
df = df.withColumn("goal_difference", df["home_scoreHome"] - df["away_scoreAway"])
maior_goleada = df.orderBy(df["goal_difference"].desc()).first()

print(f"{color.BOLD}{color.GREEN}RESPOSTA{color.END}")
df.filter((df["goal_difference"] == maior_goleada["goal_difference"])).show()

#### 10. Quantos jogos ocorreram no Brasil?

In [0]:
partidas_brasil = df.filter(df["countryCountry"] == "Brazil").count()
print(f"{color.BOLD}{color.GREEN}RESPOSTA{color.END}\nA base de dados possui {partidas_brasil} registros onde a partida foi realizada no Brasil.")